In [18]:
import sys, os
from pathlib import Path
import llm, sqlite_utils
from itertools import batched

In [19]:
print("Environment check goes brrr...")
assert(sys.executable.split('/')[-3] == 'conda-env')
assert(os.environ.get('ENVIRONMENT_LOADED') == 'roombot')

Environment check goes brrr...


In [20]:
print("Database create (or load) goes brrr...")

db_dir = "./db/"
Path(db_dir).mkdir(parents=True, exist_ok=True)
db_file = db_dir + "embeddings.db"
db = sqlite_utils.Database(db_file)

Database create (or load) goes brrr...


In [21]:
print("Embeddings collection create (or load) goes brrr...")

model_id = "3-large"
embedding_model = llm.get_embedding_model(model_id)

collection_name = "code-collection"

# if llm.Collection.exists(db, collection_name):
#     print("Deleting pre-existing embedding collection in db...")
#     collection = llm.Collection(collection_name, db, model=embedding_model)
#     collection.delete()
    
collection = llm.Collection(collection_name, db, model=embedding_model)


Embeddings collection create (or load) goes brrr...


In [22]:
print("LLM model setup goes brrr...")
llm_model = llm.get_model("gpt-4o")

LLM model setup goes brrr...


In [25]:
print("Reading code goes brrr...")

to_embed = [
    # (id, content_to_embed, metadata)
]
code_dir = os.environ['ROOMBOT_CODE_DIR'].rstrip('/') + '/'

print("Using directory: ", code_dir)
paths = Path(code_dir).glob("**/*")
num_files = len(list(filter(lambda path: path.is_file(), paths)))
paths = Path(code_dir).glob("**/*")
i = 0
for path in filter(lambda path: path.is_file(), paths):
    print(str(i+1) + " of " + str(num_files) + "...")
    code = path.read_text()
    print("LLM model used!")
    response = llm_model.prompt("You are an expert code auditor, working as a member a team. "+
                                "Your team is working on a project involving a large amount of code. "+
                                "Some of the code is known to be of poor quality in terms of long-term "+
                                "maintainability, cleanliness, and performance. "+
                                "The team's goal is to prioritize which parts of the code require immediate "+
                                "attention and refactoring. You have been given the following code file,  "+
                                "delimited between `>>>>>>>>>` and `<<<<<<<<<`"+
                                "Your task is to write a short description of the quality of the code. "+
                                "These descriptions will later be used to prioritize refactorings. The code follows now: "+
                                ">>>>>>>>>\n" + code + "\n<<<<<<<<<")
    code_description = (response.text())
    to_embed.append((i, code_description, {"path": str(path)}))
    i+=1

# print("        ...brr... using hardcoded")
# to_embed = [
#     (1, "This is really good code.", {"path": "good"}),
#     (2, "This is really bad code.", {"path": "bad"}),
# ]

Reading code goes brrr...
Using directory:  /home/ydhamija/Stuff/yawa/src/
1 of 12...
LLM model used!
2 of 12...
LLM model used!
3 of 12...
LLM model used!
4 of 12...
LLM model used!
5 of 12...
LLM model used!
6 of 12...
LLM model used!
7 of 12...
LLM model used!
8 of 12...
LLM model used!
9 of 12...
LLM model used!
10 of 12...
LLM model used!
11 of 12...
LLM model used!
12 of 12...
LLM model used!


In [26]:
print("Embedding goes brrr...")
print("Embedding model used!")
collection.embed_multi_with_metadata(to_embed, store=True)

Embedding goes brrr...
Embedding model used!


In [27]:
print("Searching embeddings goes brrr...")
query = "This code needs to be refactored."
print("Embedding model used!")
for entry in collection.similar(query, number=3):
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    print("ENTRY ID and SCORE:")
    print(entry.id, entry.score)
    print("\n\nMETADATA:")
    print(entry.metadata)
    print("\n\nEMBEDDED CONTENT:")
    print(entry.content)
    print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")

Searching embeddings goes brrr...
Embedding model used!
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
ENTRY ID and SCORE:
2 0.3818518398256749


METADATA:
{'path': '/home/ydhamija/Stuff/yawa/src/programs.rs'}


EMBEDDED CONTENT:
### Code Quality Description

Overall, the provided code is functional and has certain elements of good quality; however, there are several areas that require attention to improve maintainability, readability, and performance:

1. **Mutability and Self-consumption**:
    - The use of `mut self` in methods and `self` consumption in `complete_workout` can be error-prone and unnecessary. Functions such as `complete_workout` and some other helper methods require self-consumption, leading to ineffective state management.

2. **Error Handling**:
    - Error handling is largely absent. Methods like `start_gzcl_4day` use unwrap without considering the potential failure of parsing lifts, which could result in a panic.

